In [1]:
from datasets import load_from_disk, Dataset
from tqdm import tqdm, trange
import pandas as pd
import pickle

In [2]:
# tf-idf로 retrieval한 상위 100개의 query - context 묶음
with open('/opt/ml/data/tfidf_fit_train.bin', "rb") as file:
    tf_idf_pairs = pickle.load(file)

In [3]:
train_dataset = load_from_disk("/opt/ml/data/train_dataset/train/")

In [ ]:
tf_idf_pairs[train_dataset['question'][0]][0:5] # 확인

In [20]:
new_context = []
for i in tqdm(range(len(train_dataset))):
    train_context = train_dataset['context'][i] # ground truth
    query = train_dataset['question'][i] # query
    sim_context = tf_idf_pairs[query] # tf - idf context list
    answer = train_dataset['answers'][i]['text'][0]
    
    cnt = 4 # 추가할 갯수
    sim_context_idx = 0
    sim_top_k = [train_context]

    while cnt != 0:
        if train_context != sim_context[sim_context_idx] and not (answer in sim_context[sim_context_idx]):
            # 비슷한 context를 추가하되 정답을 포함하지 않는 문장을 추가한다.
            sim_top_k.append(sim_context[sim_context_idx])
            cnt -= 1
        sim_context_idx += 1
        if sim_context_idx == 100: # index를 넘어가면 break
            break
    add_sim_context = ' '.join(sim_top_k)
    new_context.append(add_sim_context)

100%|██████████| 3952/3952 [09:15<00:00,  7.12it/s]


In [21]:
train_df = train_dataset.to_pandas()
train_df['context'] = new_context

In [22]:
train_df['context'][0] # 확인

'미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05 국회에 관해 규정하는 헌법 제4장의 첫 조문이다.\n\n본조에서 말하는 "국권"이란 국가가 갖는 지배권을 포괄적으로 나타내는 국가 권력, 곧 국가의 통치권을 의미한다. 국권은 일반적으로 입법권·행정권·사법권의 3권으로 분류되지만, 그 중

In [23]:
# save datasets
new_train_dataset = Dataset.from_pandas(train_df)
new_train_dataset.save_to_disk('/opt/ml/data/train_dataset/tf_idf_train')
print('done')

done
